# Data Cleaning for Customer Satisfaction Analysis

This notebook is part of a larger project exploring customer satisfaction in Brazilian e-commerce using the [Olist dataset](https://www.kaggle.com/olistbr/brazilian-ecommerce).  
Here, we focus on loading, exploring, and cleaning the raw data to prepare it for further analysis.

**Goals of this notebook:**
- Load and inspect the raw datasets
- Explore key features and structure
- Handle missing values and duplicates
- Save clean, analysis-ready data for further use

For the analysis, see the [Customer Satisfaction Analysis Notebook](./02_customer-satisfaction-analysis.ipynb).

## **Structure of the Notebook**

*Note: Section links and “Back to top” links work best in Jupyter environments (e.g., Jupyter Lab or VS Code). They may not work as expected when clicked directly on GitHub.*

- [Data Overview](#data-overview)  
  - [Loading raw datasets](#loading-raw-datasets)
  - [Exploring datasets (structure, sampling, descriptive statistics)](#exploring-datasets)
- [Data Cleaning](#data-cleaning)
  - [Handling missing data](#handling-missing-data)
  - [Removing duplicates](#removing-duplicates)
- [Saving Cleaned DataFrames](#Saving-Cleaned-DataFrames)

In [7]:
# Import libraries
import os
import pandas as pd

In [8]:
# Set display options for better readability in output
#pd.set_option("display.max_colwidth", None)
pd.set_option("display.float_format", '{:,.2f}'.format)

## **Data Overview**

### Loading raw datasets

➤ Load all raw CSV files into individual DataFrames and store them in a dictionary for easier handling and quick access during exploration.

In [9]:
# Load all CSV files from the Brazilian E-Commerce dataset into separate DataFrames
data_path = "../data/raw/"
files = os.listdir(data_path)

customers_raw = pd.read_csv(data_path + "olist_customers_dataset.csv")
geolocation_raw = pd.read_csv(data_path + "olist_geolocation_dataset.csv")
orders_raw = pd.read_csv(data_path + "olist_orders_dataset.csv")
items_raw = pd.read_csv(data_path + "olist_order_items_dataset.csv")
payments_raw = pd.read_csv(data_path + "olist_order_payments_dataset.csv")
reviews_raw = pd.read_csv(data_path + "olist_order_reviews_dataset.csv")
products_raw = pd.read_csv(data_path + "olist_products_dataset.csv")
sellers_raw = pd.read_csv(data_path + "olist_sellers_dataset.csv")
translation_raw = pd.read_csv(data_path + "product_category_name_translation.csv")

# Store all DataFrames in a dictionary for easier looping and inspection
dataframes_raw = {
    "customers": customers_raw,
    "geolocation": geolocation_raw,
    "orders": orders_raw,
    "items": items_raw,
    "payments": payments_raw,
    "reviews": reviews_raw,
    "products": products_raw,
    "sellers": sellers_raw,
    "translation": translation_raw,
}


The following tables are included in the Brazilian E-Commerce dataset:

- `customers`: customer information  
- `geolocation`: geographical coordinates by zip code prefix  
- `orders`: order details including status and timestamps  
- `items`: product-level details for each order  
- `payments`: payment methods, amounts and installment information 
- `reviews`: customer reviews and ratings  
- `products`: product attributes including category and dimensions
- `sellers`: seller information  
- `translation`: Portuguese-to-English product category mapping 

*Note: Original file names such as `olist_customers_dataset.csv` were renamed to simpler identifiers like `customers` for ease of use.*

[🠉 Back to top](#structure-of-the-notebook)

### Exploring Datasets

➤  Summary of all tables using `.shape`, column names, and duplicate counts.

In [10]:
# Define a function to summarize a dictionary of DataFrames
def summarize_dataframes(df_dict=dataframes_raw):
    """
    Takes a dictionary of DataFrames and returns a summary DataFrame
        with the following information for each:
        - name: the key name from the dictionary
        - rows: number of rows in the DataFrame
        - columns: number of columns
        - column_names: a list of column names
        - duplicates: number of duplicated rows
    """
    summary = []

    # Loop over each DataFrame in the dictionary
    for name, df in df_dict.items():
        summary.append(
            {
                "name": name,
                "rows": df.shape[0],
                "columns": df.shape[1],
                "column_names": list(df.columns),
                "duplicates": df.duplicated().sum(),
            }
        )
    # Return a summary DataFrame
    return pd.DataFrame(summary)


# Call the function and display the summary of all loaded DataFrames
summarize_dataframes(dataframes_raw)

,name,rows,columns,column_names,duplicates
0,customers,99441,5,"[customer_id, customer_unique_id, customer_zip...",0
1,geolocation,1000163,5,"[geolocation_zip_code_prefix, geolocation_lat,...",261831
2,orders,99441,8,"[order_id, customer_id, order_status, order_pu...",0
3,items,112650,7,"[order_id, order_item_id, product_id, seller_i...",0
4,payments,103886,5,"[order_id, payment_sequential, payment_type, p...",0
5,reviews,99224,7,"[review_id, order_id, review_score, review_com...",0
6,products,32951,9,"[product_id, product_category_name, product_na...",0
7,sellers,3095,4,"[seller_id, seller_zip_code_prefix, seller_cit...",0
8,translation,71,2,"[product_category_name, product_category_name_...",0


➤  Quick sampling of 5 rows from each table for visual inspection

In [11]:
# Display a random sample of 5 rows from each DataFrame for a quick visual inspection
for name, df in dataframes_raw.items():
    print(f'{name.capitalize()}:')
    display(df.sample(5))
    print("-"*130)

Customers:


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
2527,560a8e01b6209b466db525b3f3bfbae2,3fed19e15a61d255abd1b1ddd4c16c2a,58013,joao pessoa,PB
31861,5b76c1a96fc01949502602e1eec0e4d9,a665220976225ee856a75ff6afe998d9,21040,rio de janeiro,RJ
58890,a95ad7bcbf76f95fcb884dfdfa84ae68,752e16eb96c1372769f734d8ae79f23e,55614,vitoria de santo antao,PE
98309,10c6c3b09d214daff1750003e2d2f64c,c2437c121c5840db3d5796efb6242aae,13208,jundiai,SP
31572,5e3a2c72ada2b3eebb4e370ef2da4073,574fed86093033e28725831cc252bcc8,5539,sao paulo,SP


----------------------------------------------------------------------------------------------------------------------------------
Geolocation:


,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
591404,35410,-20.34,-43.67,cachoeira do campo,MG
610406,36016,-21.77,-43.35,juiz de fora,MG
125547,5332,-23.55,-46.75,sao paulo,SP
120666,5133,-23.49,-46.74,são paulo,SP
523063,28940,-22.83,-42.10,sao pedro da aldeia,RJ


----------------------------------------------------------------------------------------------------------------------------------
Orders:


...

----------------------------------------------------------------------------------------------------------------------------------
Sellers:


,seller_id,seller_zip_code_prefix,seller_city,seller_state
1130,e3dd723429d1b23614b7fa85e2c7a853,9810,sao bernardo do campo,SP
940,ed4311e67debd72b8610a5347a743087,7170,guarulhos,SP
1486,340636a908b13bbe0f4fb63fbe1d0c03,12240,sao jose dos campos,SP
15,1444c08e64d55fb3c25f0f09c07ffcf2,42738,lauro de freitas,BA
605,e59e575a175fadcadf8479e191770425,26210,nova iguacu,RJ


----------------------------------------------------------------------------------------------------------------------------------
Translation:


,product_category_name,product_category_name_english
46,artes,art
21,fashion_calcados,fashion_shoes
26,moveis_cozinha_area_de_servico_jantar_e_jardim,kitchen_dining_laundry_garden_furniture
43,construcao_ferramentas_iluminacao,construction_tools_lights
50,fashion_underwear_e_moda_praia,fashion_underwear_beach


----------------------------------------------------------------------------------------------------------------------------------


➤ Column-wise overview including dtypes, missing values, and unique counts.

In [12]:
# Quick overview of column properties (dtypes, missing values, uniques) for all DataFrames
def overview(df_dict=dataframes_raw):
    """
    Creates and displays a column-wise overview for each DataFrame in a dictionary.

    Parameters:
        df_dict (dict): A dictionary of DataFrames (e.g., {'orders': orders, ...})

    Displays:
        For each DataFrame:
            - Data type
            - Non-null count
            - Missing value count and percentage
            - Missing value percentage
            - Number of unique values
            - Unique values
    """
    for name, df in df_dict.items():
        print(f'{name.capitalize()}:')
        summary = pd.DataFrame(
                {
                    "dtype": df.dtypes,
                    "total": df.count(),
                    "missing_n": df.isna().sum(),
                    "missing_%": df.isna().mean() * 100,
                    "uniques_n": df.nunique(),
                    "uniques": [df[col].unique() for col in df.columns],
                }
        )
        display(summary)   
        print("-"*130)

overview(dataframes_raw)


Customers:


,dtype,total,missing_n,missing_%,uniques_n,uniques
customer_id,object,99441,0,0.00,99441,"[06b8999e2fba1a1fbc88172c00ba8bc7, 18955e83d33..."
customer_unique_id,object,99441,0,0.00,96096,"[861eff4711a542e4b93843c6dd7febb0, 290c77bc529..."
customer_zip_code_prefix,int64,99441,0,0.00,14994,"[14409, 9790, 1151, 8775, 13056, 89254, 4534, ..."
customer_city,object,99441,0,0.00,4119,"[franca, sao bernardo do campo, sao paulo, mog..."
customer_state,object,99441,0,0.00,27,"[SP, SC, MG, PR, RJ, RS, PA, GO, ES, BA, MA, M..."


----------------------------------------------------------------------------------------------------------------------------------
Geolocation:


,dtype,total,missing_n,missing_%,uniques_n,uniques
geolocation_zip_code_prefix,int64,1000163,0,0.00,19015,"[1037, 1046, 1041, 1035, 1012, 1047, 1013, 102..."
geolocation_lat,float64,1000163,0,0.00,717360,"[-23.54562128115268, -23.54608112703553, -23.5..."
geolocation_lng,float64,1000163,0,0.00,717613,"[-46.63929204800168, -46.64482029837157, -46.6..."
geolocation_city,object,1000163,0,0.00,8011,"[sao paulo, são paulo, sao bernardo do campo, ..."
geolocation_state,object,1000163,0,0.00,27,"[SP, RN, AC, RJ, ES, MG, BA, SE, PE, AL, PB, C..."


----------------------------------------------------------------------------------------------------------------------------------
Orders:


...

----------------------------------------------------------------------------------------------------------------------------------
Sellers:


,dtype,total,missing_n,missing_%,uniques_n,uniques
seller_id,object,3095,0,0.00,3095,"[3442f8959a84dea7ee197c632cb2df15, d1b65fc7deb..."
seller_zip_code_prefix,int64,3095,0,0.00,2246,"[13023, 13844, 20031, 4195, 12914, 20920, 5532..."
seller_city,object,3095,0,0.00,611,"[campinas, mogi guacu, rio de janeiro, sao pau..."
seller_state,object,3095,0,0.00,23,"[SP, RJ, PE, PR, GO, SC, BA, DF, RS, MG, RN, M..."


----------------------------------------------------------------------------------------------------------------------------------
Translation:


,dtype,total,missing_n,missing_%,uniques_n,uniques
product_category_name,object,71,0,0.00,71,"[beleza_saude, informatica_acessorios, automot..."
product_category_name_english,object,71,0,0.00,71,"[health_beauty, computers_accessories, auto, b..."


----------------------------------------------------------------------------------------------------------------------------------


➤ Quick statistical overview of all numeric columns in each raw table to spot any unusual values or patterns.

In [13]:
# Summarize basic statistics of all numeric columns for each DataFrame in the dictionary
def describe_numeric_columns(df_dict=dataframes_raw):
    """
    Displays a transposed summary of descriptive statistics (.describe().T)
    for all numeric columns in each DataFrame within the given dictionary.

    Parameters:
    df_dict (dict): A dictionary where keys are table names and values are pandas DataFrames.

    Notes:
    - If a DataFrame has no numeric columns, a message is printed instead.
    - The output includes a visual summary using display() for easier inspection in notebooks.
    """
    for name, df in df_dict.items():
        print(f"{name.capitalize()}:")
        numeric_df = df.select_dtypes(include="number")

        if numeric_df.empty:
            print("No numeric columns to describe.")
        else:
            display(numeric_df.describe().T)

        print("-" * 130)


describe_numeric_columns()

Customers:


,count,mean,std,min,25%,50%,75%,max
customer_zip_code_prefix,"99,441.00","35,137.47","29,797.94","1,003.00","11,347.00","24,416.00","58,900.00","99,990.00"


----------------------------------------------------------------------------------------------------------------------------------
Geolocation:


,count,mean,std,min,25%,50%,75%,max
geolocation_zip_code_prefix,"1,000,163.00","36,574.17","30,549.34","1,001.00","11,075.00","26,530.00","63,504.00","99,990.00"
geolocation_lat,"1,000,163.00",-21.18,5.72,-36.61,-23.60,-22.92,-19.98,45.07
geolocation_lng,"1,000,163.00",-46.39,4.27,-101.47,-48.57,-46.64,-43.77,121.11


----------------------------------------------------------------------------------------------------------------------------------
Orders:
No numeric columns to describe.
----------------------------------------------------------------------------------------------------------------------------------
Items:


...

----------------------------------------------------------------------------------------------------------------------------------
Products:


,count,mean,std,min,25%,50%,75%,max
product_name_lenght,"32,341.00",48.48,10.25,5.00,42.00,51.00,57.00,76.00
product_description_lenght,"32,341.00",771.50,635.12,4.00,339.00,595.00,972.00,"3,992.00"
product_photos_qty,"32,341.00",2.19,1.74,1.00,1.00,1.00,3.00,20.00
product_weight_g,"32,949.00","2,276.47","4,282.04",0.00,300.00,700.00,"1,900.00","40,425.00"
product_length_cm,"32,949.00",30.82,16.91,7.00,18.00,25.00,38.00,105.00
product_height_cm,"32,949.00",16.94,13.64,2.00,8.00,13.00,21.00,105.00
product_width_cm,"32,949.00",23.20,12.08,6.00,15.00,20.00,30.00,118.00


----------------------------------------------------------------------------------------------------------------------------------
Sellers:


,count,mean,std,min,25%,50%,75%,max
seller_zip_code_prefix,"3,095.00","32,291.06","32,713.45","1,001.00","7,093.50","14,940.00","64,552.50","99,730.00"


----------------------------------------------------------------------------------------------------------------------------------
Translation:
No numeric columns to describe.
----------------------------------------------------------------------------------------------------------------------------------


[🠉 Back to top](#structure-of-the-notebook)

## **Data Cleaning**

➤ Copy raw DataFrames into a new working dictionary to preserve the original data before cleaning.

In [14]:
# Create a new dictionary with copies of all raw DataFrames
def copy_raw_dataframes(raw_dict, exclude=None):
    """
    Creates copies of raw DataFrames to preserve the original data before any cleaning steps.

    Parameters:
        raw_dict (dict): Dictionary containing raw DataFrames.
        exclude (list): List of table names to exclude from copying.

    Returns:
        dict: A new dictionary with copies of the DataFrames.
    """
    exclude = exclude or []
    copy_dict = {}

    for name, df in raw_dict.items():
        if name in exclude:
            continue
        copy_dict[name] = df.copy()

    return copy_dict


dataframes = copy_raw_dataframes(dataframes_raw, exclude=["geolocation"])  # exclude 'geolocation', which is not used in the analysis

[🠉 Back to top](#structure-of-the-notebook)

### Handling missing data

➤ Missing product-related information

In [15]:
# Copy the 'products' DataFrame from the dictionary for further processing
products = dataframes["products"].copy()

# Display the number of missing values in each column of the 'products' DataFrame
products.isna().sum()

product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64

In [16]:
# Count the number of rows where any important product-related column is missing
products_missing_cols = [
    "product_category_name",
    "product_name_lenght",
    "product_description_lenght",
    "product_photos_qty"
]

number_missing_all = products[products_missing_cols].isna().any(axis=1).sum()

# Print the number and percentage of affected rows
print(
    f"{number_missing_all} rows have missing values in key product-related columns "
    f"({number_missing_all / products.shape[0]:.1%} of the products table)."
)

610 rows have missing values in key product-related columns (1.9% of the products table).


In [17]:
# Drop rows where all key product-related columns are missing (category, name length, description length, and photo count)
products.dropna(subset=products_missing_cols, how="all", inplace=True)

# Display the number of missing values remaining in each column
products.isna().sum()

product_id                    0
product_category_name         0
product_name_lenght           0
product_description_lenght    0
product_photos_qty            0
product_weight_g              1
product_length_cm             1
product_height_cm             1
product_width_cm              1
dtype: int64

Rows missing `product_category_name` (610 in total) are dropped, as this column is critical for analyzing product-level trends in customer satisfaction. Without it, meaningful grouping and interpretation are not possible.

These rows also lack values in other key descriptive columns — such as `product_name_lenght`, `product_description_lenght`, and `product_photos_qty` — which are relevant for understanding how product presentation may affect customer perception. Since these fields are simultaneously missing, and no reliable imputation method is available, we exclude these rows entirely.

Other missing fields, such as product dimensions and weight, are not directly relevant to our current analysis. However, to ensure the dataset remains reusable for future projects, we will impute the missing values for `product_weight_g`, `product_length_cm`, `product_height_cm`, and `product_width_cm` using median values from products in the same category and a similar price range.

In [18]:
# Define the columns that describe the product's physical dimensions
size_cols = [
    "product_weight_g",
    "product_length_cm",
    "product_height_cm",
    "product_width_cm",
]

# View rows where at least one of the size columns has missing values
products[products[size_cols].isna().any(axis=1)]

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
8578,09ff539a621711667c43eba6a3bd8466,bebes,60.00,865.00,3.00,NaN,NaN,NaN,NaN


There is now only one row in the dataset with missing values (`product_id` equal to `'09ff539a621711667c43eba6a3bd8466'`), and it happens to be missing all four size-related columns. This product belongs to the `'bebe'` category. Despite this being an isolated case, we will write the following code in a generalized way to ensure it can be reused for other projects or datasets with more missing values.

In [19]:
# Merge median item price into the products table to enable price-based imputation
items = dataframes["items"].copy()
price_by_product = items.groupby("product_id")["price"].median().reset_index()
products_price = products.merge(price_by_product, on="product_id", how="left")

In [20]:
# Create price bins (quartiles) within each product category, based on median price
# This helps identify similar products by both category and price level
products_price["price_bin"] = products_price.groupby("product_category_name")[
    "price"
].transform(lambda x: pd.qcut(x, q=4, duplicates="drop"))

In [21]:
# Impute missing size values using the median for each (category, price_bin) group
for col in size_cols:
    # Calculate the group-specific median for the current column
    group_medians = products_price.groupby(["product_category_name", "price_bin"])[
        col
    ].transform("median")
    # Fill missing values in the current column using the group medians
    products_price[col] = products_price[col].fillna(group_medians)

In [22]:
# Print the number of missing values remaining in each column for verification
# Note: 'price_bin' may have missing values if a product belongs to a category with only one product,
# since quartile-based binning (qcut) cannot be applied in such cases.
print(products_price.isna().sum())
# Verify that the one product with missing values was successfully imputed
products_price[products_price["product_id"] == '09ff539a621711667c43eba6a3bd8466']

product_id                    0
product_category_name         0
product_name_lenght           0
product_description_lenght    0
product_photos_qty            0
product_weight_g              0
product_length_cm             0
product_height_cm             0
product_width_cm              0
price                         0
price_bin                     1
dtype: int64


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,price,price_bin
8412,09ff539a621711667c43eba6a3bd8466,bebes,60.00,865.00,3.00,"8,200.00",42.00,32.00,40.00,"1,934.00","(159.9, 3899.0]"


In [23]:
# Remove the helper columns and save the cleaned/imputed data back to 'products'
products = products_price.drop(columns=["price", "price_bin"])

[🠉 Back to top](#structure-of-the-notebook)

➤ Missing English translation for product category names

Another issue identified during the data overview stage is that the `products` table contains two category names that are missing from the `translation` table. To ensure consistency in our analysis, we will merge the two tables to add English category names and manually fill in the missing translations.

In [24]:
# Merge products with the translation table
translation = dataframes["translation"].copy()

products_translated = pd.merge(products, translation, how="left", on="product_category_name")

# Identify product categories that are missing English translations
categories_missing_translation = products_translated[
    products_translated["product_category_name_english"].isna()
]["product_category_name"].unique()

print(f"Categories missing English translation: {categories_missing_translation}")

Categories missing English translation: ['pc_gamer' 'portateis_cozinha_e_preparadores_de_alimentos']


The category name `'pc_gamer'` can remain unchanged.
`'portateis_cozinha_e_preparadores_de_alimentos'` refers to kitchen appliances and food preparation tools and will be translated as `'kitchen_appliances_and_food_processors'`.

In [25]:
# Manual translation for missing categories
category_translation_dict = {
    "pc_gamer": "pc_gamer",
    "portateis_cozinha_e_preparadores_de_alimentos": "kitchen_appliances_and_food_processors",
}

# Fill in missing translations using the manual dictionary
products_translated["product_category_name_english"] = products_translated[
    "product_category_name_english"
].fillna(products_translated["product_category_name"].map(category_translation_dict))


Now that we have handled the missing values in the `products` table and added English translations for all product categories, we can update the table in the dictionary. The `translation` table is no longer needed and will be removed.  
Columns related to product dimensions, weight, and the original Portuguese category names will be dropped at a later stage.

In [26]:
# Update the 'products' table in the dictionary with the translated version
dataframes["products"] = products_translated

# Remove the 'translation' table from the dictionary since it's no longer needed
del dataframes["translation"]

[🠉 Back to top](#structure-of-the-notebook)

### Removing duplicates

During the data overview stage, we found that neither `review_ids` nor `order_ids` are unique in the `reviews` table. However, for a consistent and reliable analysis of customer satisfaction, it is essential to ensure a one-to-one relationship between reviews and orders — meaning each `order_id` should be associated with exactly one `review_id`, and each `review_id` should correspond to only one `order_id`.
This prevents double-counting reviews and ensures that satisfaction scores are accurately linked to individual orders.

➤ Removing duplicates in `reviews_id` column

In [27]:
# Copy the reviews and items tables
reviews = dataframes["reviews"].copy()
items = dataframes["items"].copy()

# Filter rows where the same review_id appears more than once
duplicate_reviews = reviews[reviews.duplicated("review_id", keep=False)].copy()

# Count how many times each review_id appears
review_id_counts = duplicate_reviews["review_id"].value_counts()

# Add the count back to the duplicate_reviews dataframe
duplicate_reviews["review_id_count"] = duplicate_reviews["review_id"].map(review_id_counts)

# Merge with items table to inspect what products are tied to these duplicated reviews
duplicate_reviews_items = pd.merge(duplicate_reviews, items, how="left", on="order_id")

# Report the number of duplicated review_id entries
print(
    f"Number of duplicated review_id entries: {duplicate_reviews.shape[0]} out of {reviews.shape[0]} "
    f"({duplicate_reviews.shape[0] / reviews.shape[0]:.1%} of the reviews dataset)."
)

# Sort by frequency of review_id, then review_id and order_id for easier inspection
duplicate_reviews_items_sorted = duplicate_reviews_items.sort_values(
    ["review_id_count", "review_id", "order_id"], ascending=[False, True, True]
)

# Display top rows with the most duplicated review_ids
duplicate_reviews_items_sorted.head(15)

Number of duplicated review_id entries: 1603 out of 99224 (1.6% of the reviews dataset).


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,review_id_count,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
739,08528f70f579f0c830189efc523d2182,03310aa823a66056268a3bab36e827fb,1,Produto errado,Entrega do produto diferente do solicitado\r\n...,2018-08-03 00:00:00,2018-08-06 00:09:52,3,NaN,NaN,NaN,NaN,NaN,NaN
1420,08528f70f579f0c830189efc523d2182,53c71d3953507c6239ff73917ed358c9,1,Produto errado,Entrega do produto diferente do solicitado\r\n...,2018-08-03 00:00:00,2018-08-06 00:09:52,3,1.00,17d98fc630d23a628ec1130f8d69d1d2,f9244d45189d3a3605499abddeade7d5,2018-08-13 20:55:31,"3,109.99",74.35
137,08528f70f579f0c830189efc523d2182,7813842ae95e8c497fc0233232ae815a,1,Produto errado,Entrega do produto diferente do solicitado\r\n...,2018-08-03 00:00:00,2018-08-06 00:09:52,3,NaN,NaN,NaN,NaN,NaN,NaN
1053,0c76e7a547a531e7bf9f0b99cba071c1,16cc0fe71527d13426bdbe29205b2053,5,NaN,NaN,2017-08-31 00:00:00,2017-09-05 15:27:17,3,1.00,5c821d665e58b08092e830f81c12cc37,abc015ebac1d283a323ca9db35ede4b9,2017-09-01 20:44:59,89.00,12.12
372,0c76e7a547a531e7bf9f0b99cba071c1,3525e0e57f9d276d522d570bd46cb39c,5,NaN,NaN,2017-08-31 00:00:00,2017-09-05 15:27:17,3,1.00,fba6f88e10230dd0ba718aec962064ae,d91fb3b7d041e83b64a00a3edfb37e4f,2017-09-01 20:44:58,18.00,58.90
626,0c76e7a547a531e7bf9f0b99cba071c1,98c977c116f7779360e9fecffd3860b6,5,NaN,NaN,2017-08-31 00:00:00,2017-09-05 15:27:17,3,1.00,e4aa3f3e3209009166c6b79afa8204bc,1127b7f2594683f2510f1c2c834a486b,2017-09-01 20:44:54,34.00,9.34
891,1fb4ddc969e6bea80e38deec00393a6f,3c1098cb17277b62cfc709c7a9b500f5,5,NaN,NaN,2017-08-11 00:00:00,2017-08-12 14:35:35,3,1.00,d800be06cad2262f48d1c2982fd31189,cfb1a033743668a192316f3c6d1d2671,2017-08-02 17:31:43,53.90,17.63
885,1fb4ddc969e6bea80e38deec00393a6f,afed4265a8b956d840bc032e54dfccd1,5,NaN,NaN,2017-08-11 00:00:00,2017-08-12 14:35:35,3,1.00,45a19bf9dddef525a5881b62c01f164c,004c9cd9d87a3c30c522c48c4fc07416,2017-08-03 17:31:43,82.99,17.83
414,1fb4ddc969e6bea80e38deec00393a6f,d6dde74bdeb424af6b660214881b4845,5,NaN,NaN,2017-08-11 00:00:00,2017-08-12 14:35:35,3,1.00,2475f9f6b36857e1a35292cf761ffa48,4a3ca9315b744ce9f8e9374361493884,2017-08-01 17:31:40,47.60,5.06
415,1fb4ddc969e6bea80e38deec00393a6f,d6dde74bdeb424af6b660214881b4845,5,NaN,NaN,2017-08-11 00:00:00,2017-08-12 14:35:35,3,2.00,548e5bfe28edceab6b51fa707cc9556f,da8622b14eb17ae2831f4ac5b9dab84a,2017-08-01 17:31:40,89.90,27.84


The table above shows examples of `review_id`s that appear multiple times, each linked to a different `order_id`. These cases likely involve multi-item purchases — either of the same or different products — that should have been recorded under a single `order_id`, but were incorrectly split while sharing the same review.

Since this duplication affects only 1.6% of the reviews and introduces ambiguity into the mapping between `orders` and `reviews`, we consider it safer to remove these inconsistent entries from the dataset to preserve the integrity of the analysis.

In [28]:
# Identify review_ids that appear more than once
duplicated_ids = reviews["review_id"][reviews["review_id"].duplicated(keep=False)]

# Remove rows with duplicated review_ids
reviews_no_duplicated_review_id = reviews[~reviews["review_id"].isin(duplicated_ids)]

# Print summary
print(
    f"Number of rows in updated reviews table: {reviews_no_duplicated_review_id.shape[0]}"
)
print(
    f"Number of unique review_ids: {reviews_no_duplicated_review_id['review_id'].nunique()}"
)


Number of rows in updated reviews table: 97621
Number of unique review_ids: 97621


[🠉 Back to top](#structure-of-the-notebook)

➤ Removing duplicates in `order_id` column

In [29]:
# Filter rows where the same order_id appears more than once
duplicate_orders = reviews_no_duplicated_review_id[
    reviews_no_duplicated_review_id.duplicated("order_id", keep=False)
].copy()

# Count how many times each order_id appears
order_id_counts = duplicate_orders["order_id"].value_counts()

# Add the count as a new column for easier inspection
duplicate_orders["order_id_count"] = duplicate_orders["order_id"].map(order_id_counts)

# Merge with items table to inspect which products are tied to these duplicated orders
duplicate_orders_items = pd.merge(duplicate_orders, items, how="left", on="order_id")

# Report the number of duplicated order_id entries
print(
    f"Number of duplicated order_id entries: {duplicate_orders.shape[0]} out of {reviews_no_duplicated_review_id.shape[0]} "
    f"({duplicate_orders.shape[0] / reviews_no_duplicated_review_id.shape[0]:.1%} of the reviews dataset after removing review_id duplicates)."
)

# Sort by frequency of order_id, then by order_id and review_id
duplicate_orders_items_sorted = duplicate_orders_items.sort_values(
    ["order_id_count", "order_id", "review_id"], ascending=[False, True, True]
)

# Display top rows with the most duplicated order_ids
duplicate_orders_items_sorted.head(15)

Number of duplicated order_id entries: 182 out of 97621 (0.2% of the reviews dataset after removing review_id duplicates).


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,order_id_count,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
44,2eab0b2e6f5bded4d9b0b2afcfdf4534,02e0b68852217f5715fb9cc885829454,4,NaN,Gostei chegou rapidinho,2017-12-03 00:00:00,2017-12-03 21:56:37,2,1,2028bf1b01cafb2d2b1901fca4083222,cc419e0650a3c5ba77189a1882b7556a,2017-12-05 06:31:54,56.99,11.90
88,fa493ead9b093fb0fa6f7d4905b0ef3b,02e0b68852217f5715fb9cc885829454,4,NaN,Gostei e entregou rapidinho,2017-12-01 00:00:00,2017-12-03 21:57:31,2,1,2028bf1b01cafb2d2b1901fca4083222,cc419e0650a3c5ba77189a1882b7556a,2017-12-05 06:31:54,56.99,11.90
160,405eb2ea45e1dbe2662541ae5b47e2aa,03c939fd7fd3b38f8485a0f95798f1f6,3,NaN,Seria ótimo se tivesem entregue os 3 (três) pe...,2018-03-06 00:00:00,2018-03-06 19:50:32,2,1,ab5da1daa941470d14366f4e76a99dd2,d91fb3b7d041e83b64a00a3edfb37e4f,2018-03-02 09:35:46,26.00,16.11
26,b04ed893318da5b863e878cd3d0511df,03c939fd7fd3b38f8485a0f95798f1f6,3,NaN,Um ponto negativo que achei foi a cobrança de ...,2018-03-20 00:00:00,2018-03-21 02:28:23,2,1,ab5da1daa941470d14366f4e76a99dd2,d91fb3b7d041e83b64a00a3edfb37e4f,2018-03-02 09:35:46,26.00,16.11
103,c793b25686e244588d4fc71964c95a2c,04f1827088d972a62224f5203a071500,1,NaN,NaN,2018-01-07 00:00:00,2018-01-07 11:14:04,2,1,daf9301b3e3756d45250765852ed0dfd,004c9cd9d87a3c30c522c48c4fc07416,2017-12-18 20:50:26,159.69,18.37
107,f543cfa959c3289703f144a32113bd78,04f1827088d972a62224f5203a071500,5,NaN,NaN,2017-12-30 00:00:00,2018-01-03 10:40:06,2,1,daf9301b3e3756d45250765852ed0dfd,004c9cd9d87a3c30c522c48c4fc07416,2017-12-18 20:50:26,159.69,18.37
135,7a49d6129a979bbbb500cb14853d2714,056bfadd41b8600ad5ecfef2ac132188,4,NaN,NaN,2017-12-05 00:00:00,2017-12-09 13:53:55,2,1,2bb67f2cd3ff60400e0bad33aee72d2a,daeb5653dd96c1b11860f72209795012,2017-11-30 11:15:31,19.90,15.11
68,acf3f5b4d93975335ee6a76654cd495b,056bfadd41b8600ad5ecfef2ac132188,4,NaN,NaN,2017-12-05 00:00:00,2017-12-09 13:53:28,2,1,2bb67f2cd3ff60400e0bad33aee72d2a,daeb5653dd96c1b11860f72209795012,2017-11-30 11:15:31,19.90,15.11
101,1b729a0ff69b1b85e6105a45072cdf9e,0749426d1c48fe5943cbdf1316ace0aa,1,NaN,Um década pra entregar o produto. Horrível a e...,2018-02-09 00:00:00,2018-02-10 12:55:50,2,1,9bb8ca338e5588c361e34eae02e8fad6,620c87c171fb2a6dd6e8bb4dec959fc6,2018-01-19 13:33:29,69.90,17.74
93,6d9236aeca36f0a36200fdd41e61e28c,0749426d1c48fe5943cbdf1316ace0aa,1,NaN,Pessima entrega demora um século pra entregar,2018-02-10 00:00:00,2018-02-10 12:48:27,2,1,9bb8ca338e5588c361e34eae02e8fad6,620c87c171fb2a6dd6e8bb4dec959fc6,2018-01-19 13:33:29,69.90,17.74


In many cases, the same `order_id` is associated with multiple `review_id`s, which might differ in content or score. These appear to be edited versions of the same customer review, mistakenly recorded as separate entries.
To ensure that each order is represented by only one review, we keep only the latest available review per `order_id`, based on the `review_answer_timestamp` field, which offers the most precise timestamp available.

In [30]:
# Convert review_answer_timestamp to datetime
reviews_no_duplicated_review_id.loc[:, "review_answer_timestamp"] = pd.to_datetime(
    reviews_no_duplicated_review_id["review_answer_timestamp"], errors="coerce"
)

In [31]:
# Sort and keep the latest review per order_id
reviews_cleaned = reviews_no_duplicated_review_id.sort_values(
    "review_answer_timestamp"
).drop_duplicates(subset="order_id", keep="last")

In [32]:
# Print total number of rows and number of unique review_ids and order_ids in cleaned table
print(f"Number of rows in cleaned reviews table: {reviews_cleaned.shape[0]}")
print("Number of unique review_ids and order_ids:")
print(reviews_cleaned[['review_id', "order_id"]].nunique())


Number of rows in cleaned reviews table: 97530
Number of unique review_ids and order_ids:
review_id    97530
order_id     97530
dtype: int64


Now that both `review_id` and `order_id` columns are unique, we can update the `dataframes` dictionary with the `reviews_cleaned` table.

In [33]:
# Update the 'reviews' table in the dictionary with the cleaned version
dataframes["reviews"] = reviews_cleaned

[🠉 Back to top](#structure-of-the-notebook)

## **Saving Cleaned DataFrames**

In [34]:
# Save cleaned dataframes as .csv files
save_path = (
    "c:\\Users\\Olesya\\Документы\\Projects\\stackfuelpp\\data\\cleaned"
)

for name, df in dataframes.items():
    file_path = os.path.join(save_path, f"{name}.csv")
    df.to_csv(file_path, index=False)